In [1]:
# Checks if the code is in a colab notebook

try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

Run on Colab notebook

In [2]:
if IN_COLAB:
  !pip install basicsr
  drive.mount('/content/drive/')
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_HR.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_LR_clean.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_train_LR_clean.zip
  #!unzip /content/drive/MyDrive/Datasets/DIV2K_train_HR.zip
  !unzip /content/drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/gen_images_0.zip
  FOLDER_LR_TEST = 'DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'DIV2K_train_LR_clean'
  #FOLDER_HR_TRAIN = 'DIV2K_train_HR'
  FOLDER_GEN_IMAGES = 'gen_images_0'

  STUDENT_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.pth'
  STUDENT_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.csv'
  GENERATOR_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.pth'
  GENERATOR_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.csv'
  DISCRIMINATOR_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/discriminator.pth'
  DISCRIMINATOR_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/Pruning/80_percent/discriminator.csv'
  
  PRUNED_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\son.pth'
  FINE_TUNED_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\son_fine.pth'

  TEACHER_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/ESRGAN_models/RealESRGAN_x4plus.pth'

Run on my Windows desktop

In [3]:
if not IN_COLAB:
  FOLDER_LR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_LR_clean'
  FOLDER_HR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_HR'
  
  STUDENT_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.pth'
  STUDENT_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.csv'
  GENERATOR_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.pth'
  GENERATOR_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.csv'
  DISCRIMINATOR_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\discriminator.pth'
  DISCRIMINATOR_MODEL_PATH_24 = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\discriminator_24.pth'
  DISCRIMINATOR_RECORDS_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\discriminator.csv'
  FOLDER_GEN_IMAGES = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\gen_images_0'
  
  PRUNED_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\son.pth'
  FINE_TUNED_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\Pruning\80_percent\son_fine.pth'
  
  TEACHER_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\KD\ESRGAN_models\RealESRGAN_x4plus.pth'

# Setup

In [4]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import cv2
import csv
import math
import numpy as np
import os
import queue
import threading
from basicsr.archs.rrdbnet_arch import RRDBNet
import torchvision
from os import listdir, environ, path
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models import vgg19

# Making sure to use the gpu, if available
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_device(torch.device(0))

C:\Users\Vincent\anaconda3\envs\ai-robotics\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels: int, out_channels: int, kernel_size: int, use_act: bool, **kwargs):
    super().__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
    self.activation = nn.LeakyReLU(.2, inplace=True) if use_act else nn.Identity()

  def forward(self, x):
    return self.activation(self.conv(x))

class RDB(nn.Module):
  def __init__(self, in_channels, middle_channels = 32, residual_scale = .2):
    super().__init__()
    self.residual_scale = residual_scale
    self.block = nn.ModuleList([ConvBlock(in_channels + i * middle_channels,
                                  middle_channels if i<4 else in_channels,
                                  3,
                                  stride=1,
                                  padding=1,
                                  use_act=i<4) for i in range(5)])


  def forward(self, x):
    input = x
    for conv in self.block:
      out = conv(input)
      input = torch.cat([input, out], dim=1)
    return self.residual_scale * out + x

class RRDB(nn.Module):
  def __init__(self, in_channels, mid_channels = 32, residual_scale = .2):
    super().__init__()
    self.residual_scale = residual_scale
    self.model = nn.Sequential(*[RDB(in_channels, middle_channels = mid_channels, residual_scale = residual_scale) for _ in range(3)])
  
  def forward(self, x):
    return self.model(x) * self.residual_scale + x

class Head(nn.Module):
  def __init__(self, out_channels = 64) -> None:
    super().__init__()
    self.model = nn.Conv2d(3, out_channels, 3, stride=1, padding=1)

  def forward(self, x):
    return self.model(x)

class Tail(nn.Module):
  def __init__(self, in_channels = 64) -> None:
    super().__init__()
    self.model = nn.Sequential(nn.Conv2d(in_channels, 256, 3, stride=1, padding=1),
                               nn.Upsample(scale_factor=4, mode='nearest'),
                               nn.LeakyReLU(.2, inplace=True),
                               nn.Conv2d(256, 3, 3, stride=1, padding=1))
  
  def forward(self, x):
    return self.model(x)

# Original code for ESRGAN:
# https://github.com/xinntao/ESRGAN/blob/master/RRDBNet_arch.py

import functools
import torch
import torch.nn as nn
import torch.nn.functional as F


def make_layer(block, n_layers, **kwargs):
    layers = []
    for _ in range(n_layers):
        layers.append(block(**kwargs))
    return nn.Sequential(*layers)


class ResidualDenseBlock_5C(nn.Module):
    def __init__(self, nf=64, gc=32, bias=True):
        super(ResidualDenseBlock_5C, self).__init__()
        # gc: growth channel, i.e. intermediate channels
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=bias)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=bias)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=bias)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=bias)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=bias)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        # initialization
        # mutil.initialize_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x5 * 0.2 + x


class RRDB(nn.Module):
    '''Residual in Residual Dense Block'''

    def __init__(self, nf, gc=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nf, gc)
        self.RDB2 = ResidualDenseBlock_5C(nf, gc)
        self.RDB3 = ResidualDenseBlock_5C(nf, gc)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out * 0.2 + x


class RRDBNet(nn.Module):
    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=23, gc=32):
        super(RRDBNet, self).__init__()

        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)
        self.RRDB_trunk = make_layer(RRDB, nb, nf=nf, gc=gc)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        #### upsampling
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        feat = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(feat))
        feat = feat + trunk

        feat = self.lrelu(self.upconv1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.upconv2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.HRconv(feat)))

        return out

In [13]:
class DIV2KLoader(object):
  def __init__(self,  high_res_folder, low_res_size: int = 48):
    self.high_res_folder = high_res_folder
    self.target_names = sorted(listdir(high_res_folder))
    self.len = len(self.target_names)
    self.crop_size = low_res_size*4
    self.resize = torchvision.transforms.Resize(low_res_size)

  
  def __getitem__(self, i):
    # Get the right target image
    target = Image.open(Path(self.high_res_folder).joinpath(self.target_names[i]))
    target = torchvision.transforms.ToTensor()(target)
    
    # Crop of target image
    x, y, h, w = torchvision.transforms.RandomCrop.get_params(target, output_size=(self.crop_size, self.crop_size))
    target = torchvision.transforms.functional.crop(target, x, y, h, w)
    
    # resizing for input image
    img = self.resize(target)
    
    return img, target


  def __len__(self):
    return self.len
  
  def restrict_size(self, size):
    if size < len(self.target_names) and size > 0:
      self.len = size
    else:
      self.len = len(self.img_names)
      print(f"Size must be between 0 and {len(self.img_names)}")

In [26]:
class VGGLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss = nn.L1Loss()
        self.vgg = vgg19(weights='DEFAULT').features[:35].eval().to(DEVICE)
        
        for param in self.vgg.parameters():
            param.requires_grad = False
    
    def forward(self, X, y):
        vgg_input_features = self.vgg(X)
        vgg_target_features = self.vgg(y)
        return self.loss(vgg_input_features, vgg_target_features)
    
class Trainer:
    def __init__(self, model):
        self.model = model
        self.opt = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.loss_fn = VGGLoss()
        self.cur_epoch = 0
        self.writer = SummaryWriter(f'runs/ESRGAN_Scratch/tensorboard')

    def run(self, dataloader, epochs, path_record: str = None, path_model: str = None, load_epoch: bool = False):
        # Initializes the records file if not existant

        if path_record and not Path(path_record).is_file():
            self.initialize_csv(path_record)
            self.cur_epoch = 0
        elif load_epoch:
            # Loads the next epoch to continue training
            self.load_cur_epoch(path_record)

        while self.cur_epoch < epochs:
            print(f'Epoch: {self.cur_epoch}')
            loss = self.train_one_epoch(dataloader)
            print('****************************************\n')
            print(f'Total Loss: {loss}')

            self.writer.add_scalar('Training Loss - Epoch', loss, global_step = self.cur_epoch)
            
            if path_record:
                self.record(path_record, [self.cur_epoch, loss])
            if path_model:
                self.save(path_model)
            
            self.cur_epoch += 1

    def load_cur_epoch(self, path):
        with open(path) as records_file:
            epoch = records_file.readlines()[-1].split(',')[0]
            self.cur_epoch = int(epoch) + 1

    def load(self, path):
        saved_info = torch.load(path, map_location=DEVICE)
        self.model.load_state_dict(saved_info['model_state_dict'])
        self.opt = torch.optim.Adam(self.model.parameters(), lr=saved_info['lr'])
        self.cur_epoch = saved_info['epoch'] + 1
        # self.scheduler.load_state_dict(saved_info['lr_scheduler_state_dict'])
        self.loss_fn = saved_info['loss_fn']
        print("Model loaded successfully!")

    def save(self, path):
        torch.save({'model_state_dict': self.model.state_dict(),
                'lr': self.opt.param_groups[0]['lr'],
                'loss_fn': self.loss_fn,
                'epoch': self.cur_epoch,
                'lr_scheduler_state_dict': 0},
               path)
        print("Model saved successfully!")

    def initialize_csv(self, path):
        self.record(path, ['Epoch', 'Loss'])
        print("Records created Successfully!")

    def record(self, path, row):
        with open(path, 'a', newline='') as f:
            csv.writer(f).writerow(row)
        print("Model recorded successfully!")

    def train_one_epoch(self, dataloader):
        cumu_loss = 0
        self.model.train()
        self.model.to(DEVICE)
        for batch, (X, y) in enumerate(dataloader):
            out = self.model(X.to(DEVICE))
            loss = self.learn(out, y)
            cumu_loss += loss

            if batch % 1 == 0:
                print('------------------------------------------')
                print(
                    f'Training batch {batch} with loss {loss:.5f}')
            
            # Plot metrics to tensorboard
            self.writer.add_scalar('Training Loss - Batches', loss, global_step = self.cur_epoch * len(dataloader) + batch)
            out_grid = torchvision.utils.make_grid(out.clamp_(0.,1.))
            self.writer.add_image(f'Images batch {batch}', out_grid, global_step=self.cur_epoch)
            
        return cumu_loss / len(dataloader)

    def learn(self, out, y):
        loss = self.loss_fn(out, y.to(DEVICE))

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

        return loss.item()

# Training

In [31]:
if IN_COLAB:
    RECORDS_PATH = f'drive/MyDrive/ML/Indiv_Project/Second_Year/ESRGAN_Scratch/training.csv'
    MODEL_PATH = f'drive/MyDrive/ML/Indiv_Project/Second_Year/ESRGAN_Scratch/model.pth'
else:
    RECORDS_PATH = f'D:\oldDrive\ML\Indiv_Project\Second_Year\ESRGAN_Scratch\\training.csv'
    MODEL_PATH = f'D:\oldDrive\ML\Indiv_Project\Second_Year\ESRGAN_Scratch\model.pth'

model = RRDBNet()
trainer = Trainer(model)

data_train = DIV2KLoader(FOLDER_HR_TRAIN)
data_train.restrict_size(400)
dl_train = DataLoader(data_train, batch_size=4)

In [ ]:
trainer.run(dl_train, 1000, path_record=RECORDS_PATH, path_model=MODEL_PATH)

Model recorded successfully!
Records created Successfully!
Epoch: 0
------------------------------------------
Training batch 0 with loss 1.11821
------------------------------------------
Training batch 1 with loss 1.53194
------------------------------------------
Training batch 2 with loss 1.22177
------------------------------------------
Training batch 3 with loss 1.24798
------------------------------------------
Training batch 4 with loss 1.22632
------------------------------------------
Training batch 5 with loss 1.77997
------------------------------------------
Training batch 6 with loss 1.22430
------------------------------------------
Training batch 7 with loss 1.65233
------------------------------------------
Training batch 8 with loss 1.18763
------------------------------------------
Training batch 9 with loss 1.78387
------------------------------------------
Training batch 10 with loss 1.16320
------------------------------------------
Training batch 11 with loss 1.

------------------------------------------
Training batch 2 with loss 1.08733
------------------------------------------
Training batch 3 with loss 1.07251
------------------------------------------
Training batch 4 with loss 0.96057
------------------------------------------
Training batch 5 with loss 1.21128
------------------------------------------
Training batch 6 with loss 1.09274
------------------------------------------
Training batch 7 with loss 1.17182
------------------------------------------
Training batch 8 with loss 0.75072
------------------------------------------
Training batch 9 with loss 0.88487
------------------------------------------
Training batch 10 with loss 0.93780
------------------------------------------
Training batch 11 with loss 0.95970
------------------------------------------
Training batch 12 with loss 0.94801
------------------------------------------
Training batch 13 with loss 0.71684
------------------------------------------
Training batch 14

------------------------------------------
Training batch 5 with loss 1.18697
------------------------------------------
Training batch 6 with loss 0.76302
------------------------------------------
Training batch 7 with loss 1.20620
------------------------------------------
Training batch 8 with loss 1.02562
------------------------------------------
Training batch 9 with loss 1.28222
------------------------------------------
Training batch 10 with loss 0.78138
------------------------------------------
Training batch 11 with loss 0.72147
------------------------------------------
Training batch 12 with loss 1.11710
------------------------------------------
Training batch 13 with loss 1.02136
------------------------------------------
Training batch 14 with loss 0.85113
------------------------------------------
Training batch 15 with loss 0.87169
------------------------------------------
Training batch 16 with loss 0.91056
------------------------------------------
Training batch

------------------------------------------
Training batch 8 with loss 0.84360
------------------------------------------
Training batch 9 with loss 0.83572
------------------------------------------
Training batch 10 with loss 1.14172
------------------------------------------
Training batch 11 with loss 0.79731
------------------------------------------
Training batch 12 with loss 1.19488
------------------------------------------
Training batch 13 with loss 1.28861
------------------------------------------
Training batch 14 with loss 0.73137
------------------------------------------
Training batch 15 with loss 0.57728
------------------------------------------
Training batch 16 with loss 0.79324
------------------------------------------
Training batch 17 with loss 1.26668
------------------------------------------
Training batch 18 with loss 1.05498
------------------------------------------
Training batch 19 with loss 1.04441
------------------------------------------
Training ba

------------------------------------------
Training batch 11 with loss 0.92510
------------------------------------------
Training batch 12 with loss 1.06870
------------------------------------------
Training batch 13 with loss 1.00006
------------------------------------------
Training batch 14 with loss 0.67499
------------------------------------------
Training batch 15 with loss 0.75646
------------------------------------------
Training batch 16 with loss 0.56404
------------------------------------------
Training batch 17 with loss 1.15090
------------------------------------------
Training batch 18 with loss 0.86105
------------------------------------------
Training batch 19 with loss 0.73612
------------------------------------------
Training batch 20 with loss 1.20297
------------------------------------------
Training batch 21 with loss 0.80669
------------------------------------------
Training batch 22 with loss 1.20758
------------------------------------------
Training 

...

------------------------------------------
Training batch 56 with loss 0.43061
------------------------------------------
Training batch 57 with loss 0.67762
------------------------------------------
Training batch 58 with loss 0.27717
------------------------------------------
Training batch 59 with loss 1.05559
------------------------------------------
Training batch 60 with loss 0.53438
------------------------------------------
Training batch 61 with loss 0.86167
------------------------------------------
Training batch 62 with loss 0.76319
------------------------------------------
Training batch 63 with loss 0.68706
------------------------------------------
Training batch 64 with loss 0.44080
------------------------------------------
Training batch 65 with loss 0.57228
------------------------------------------
Training batch 66 with loss 0.59820
------------------------------------------
Training batch 67 with loss 0.75561
------------------------------------------
Training 

------------------------------------------
Training batch 59 with loss 0.85351
------------------------------------------
Training batch 60 with loss 0.89104
------------------------------------------
Training batch 61 with loss 0.92880
------------------------------------------
Training batch 62 with loss 0.58238
------------------------------------------
Training batch 63 with loss 0.90038
------------------------------------------
Training batch 64 with loss 0.50423
------------------------------------------
Training batch 65 with loss 0.67415
------------------------------------------
Training batch 66 with loss 0.26781
------------------------------------------
Training batch 67 with loss 0.66717
------------------------------------------
Training batch 68 with loss 0.79381
------------------------------------------
Training batch 69 with loss 0.53597
------------------------------------------
Training batch 70 with loss 0.64929
------------------------------------------
Training 

------------------------------------------
Training batch 62 with loss 0.70298
------------------------------------------
Training batch 63 with loss 0.46790
------------------------------------------
Training batch 64 with loss 0.77354
------------------------------------------
Training batch 65 with loss 0.87143
------------------------------------------
Training batch 66 with loss 0.74145
------------------------------------------
Training batch 67 with loss 0.94395
------------------------------------------
Training batch 68 with loss 0.83826
------------------------------------------
Training batch 69 with loss 0.46365
------------------------------------------
Training batch 70 with loss 0.83406
------------------------------------------
Training batch 71 with loss 1.19612
------------------------------------------
Training batch 72 with loss 0.97045
------------------------------------------
Training batch 73 with loss 0.47664
------------------------------------------
Training 

------------------------------------------
Training batch 65 with loss 0.61961
------------------------------------------
Training batch 66 with loss 0.84293
------------------------------------------
Training batch 67 with loss 0.69787
------------------------------------------
Training batch 68 with loss 0.84709
------------------------------------------
Training batch 69 with loss 0.61825
------------------------------------------
Training batch 70 with loss 0.92331
------------------------------------------
Training batch 71 with loss 1.08433
------------------------------------------
Training batch 72 with loss 0.87655
------------------------------------------
Training batch 73 with loss 0.64161
------------------------------------------
Training batch 74 with loss 1.01671
------------------------------------------
Training batch 75 with loss 0.86542
------------------------------------------
Training batch 76 with loss 0.92655
------------------------------------------
Training 

------------------------------------------
Training batch 68 with loss 0.89828
------------------------------------------
Training batch 69 with loss 0.71071
------------------------------------------
Training batch 70 with loss 0.75061
------------------------------------------
Training batch 71 with loss 0.80523
------------------------------------------
Training batch 72 with loss 1.02455
------------------------------------------
Training batch 73 with loss 0.82224
------------------------------------------
Training batch 74 with loss 1.11990
------------------------------------------
Training batch 75 with loss 0.97948
------------------------------------------
Training batch 76 with loss 0.59972
------------------------------------------
Training batch 77 with loss 0.76717
------------------------------------------
Training batch 78 with loss 0.56173
------------------------------------------
Training batch 79 with loss 0.78738
------------------------------------------
Training 

In [23]:
trainer.writer.add_scalar('Test', 12, 0)